In [1]:
import geopandas as gpd
import shapely
import shapely
import rasterio
import glob
import pandas as pd
import numpy as np
from pathlib import Path
import sys
sys.path.append("..")
import datetime
import profiling_tools
import altair as alt
from altair import datum
import os
import json
import math

# Inputs

Provide:

* Input file path to file with cross section lines/polygons to extract low points/stream profile from
* Output file path where low points will be saved
* Input directory path to location of DEMs
* Parameter LINE_COMPLEXITY which is the number of points that each cross-section line is split into. LINE_COMPLEXITY elevation points will be extracted from the DEM for each cross section line

If you use the arg, you must run from CLI like this

```
HSFM_GEOMORPH_INPUT_FILE='inputs/mazama_inputs.json' jupyter nbconvert --execute --to html dem-analysis/mt_baker_mass_wasted/xsections.ipynb  --output outputs/xsections_mazama.html
```

In [2]:
# json_file_path = 'inputs/rainbow_inputs.json'
# Or set an env arg:
json_file_path = os.environ['HSFM_GEOMORPH_INPUT_FILE']

In [3]:
with open(json_file_path, 'r') as j:
     params = json.loads(j.read())

In [4]:
params

{'inputs': {'valley_name': 'Rainbow',
  'TO_DROP': ['1947_09_14',
   '1970_09_09',
   '1979_10_06',
   '1987_08_21',
   '1992_09_15',
   '1992_09_18'],
  'TO_DROP_LARGER_AREA': ['1947_09_14',
   '1970_09_09',
   '1974_08_10',
   '1979_10_06',
   '1987_08_21',
   '1991_09_09',
   '1992_09_15',
   '1992_09_18'],
  'erosion_polygon_file': '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/erosion.geojson',
  'erosion_by_date_polygon_file': None,
  'glacier_polygons_file': '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/glaciers.geojson',
  'dems_path': '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/dems',
  'valley_bounds_file': '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/valley_bounds.geojson',
  'plot_output_dir': '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/plots/',
  'uncertainty_file': 'outputs/uncertainty_rainbow.pcl',
  'uncertainty_file_largearea': 'outputs/uncertainty_largearea_rainbow.pcl',

In [5]:
TO_DROP = params['inputs']['TO_DROP']
input_xsections_file = params['xsections']['input_xsections_file']
output_lowpoints_file = params['xsections']['output_lowpoints_file']
output_streamlines_file = params['xsections']['output_streamlines_file']
input_dems_path = params['inputs']['dems_path']
glacier_polygons_file = params['inputs']['glacier_polygons_file']
LINE_COMPLEXITY = params['xsections']['line_complexity']

group_slope_meters = params['xsections']['group_slope_meters']

# Used to strip date from dem file names
strip_time_format = params['inputs']['strip_time_format']

reference_dem_date = datetime.datetime.strptime(
    params['inputs']['reference_dem_date'], 
    strip_time_format
)

In [6]:
raster_fns = glob.glob(os.path.join(input_dems_path, "*.tif"))
raster_fns = [fn for fn in raster_fns if Path(fn).stem not in TO_DROP]
raster_fns

['/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/dems/1970_09_29.tif',
 '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/dems/1974_08_10.tif',
 '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/dems/1977_09_27.tif',
 '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/dems/1991_09_09.tif',
 '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/dems/2013_09_13.tif',
 '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/dems/2015_09_01.tif',
 '/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/dems/2019_10_11.tif']

# Extract profiles from DEMs

Along each cross-section, extract point with lowest elevation and calculate "path distance", the distance from the furthest downstream cross section line.

In [7]:
# read cross sections file into GeoDataframe
gdf = gpd.read_file(input_xsections_file)
# Increase the number of points in each line
gdf.geometry = gdf.geometry.apply(lambda g: profiling_tools.increase_line_complexity(g, LINE_COMPLEXITY))
# Find the centroid of each line
gdf['centroid'] = gdf.geometry.apply(lambda x: x.centroid)
# Get all points from the cross section lines and create a row for each point. 
gdf['coords'] = gdf.geometry.apply(lambda x: list(x.coords))
gdf = gdf.explode('coords', ignore_index=True)
# Make the coords column a shapely.geometry.Point type and drop the cross section geometries which we no longer need.
gdf['coords'] = gdf['coords'].apply(shapely.geometry.Point)
gdf.drop(columns=["geometry"])

combined_gdf = gpd.GeoDataFrame()

for raster in raster_fns:
    print(raster)
    # Extract an elevation value for each point
    with rasterio.open(raster) as src:
        new_gdf = gdf.copy()
        # pnts['values'] = [sample[0] for sample in src.sample(coords)]
        # gdf['elevation'] = gdf['coords'].apply(lambda x: [sample for sample in src.sample(x)])
        new_gdf['elevation'] = pd.Series([sample[0] for sample in src.sample(new_gdf["coords"].apply(lambda x: x.xy))])
        new_gdf['elevation'] = new_gdf['elevation'].apply(lambda x: np.nan if x == src.nodata else x)
        
    # Convert file name to datetime as per the provided format
    date = datetime.datetime.strptime(Path(raster).stem, strip_time_format)
    new_gdf['time'] = date

    # Find the point in each cross section line (identified by the ID column, with 0 meaning furthest downstream) with the lowest elevation
    new_gdf = new_gdf.sort_values('elevation').groupby('id').apply(pd.DataFrame.head, n=1)
    new_gdf['low_point_coords'] = new_gdf.apply(lambda row: None if np.isnan(row['elevation']) else row['coords'], axis=1)

    # Set the geometry to the centroid (of the cross-section lines) to calculate "path distance", distance upstream from the furthest downstream cross-section
    new_gdf.geometry = new_gdf["centroid"]
    new_gdf['path_distance'] = pd.Series(new_gdf.distance(
            gpd.GeoDataFrame(new_gdf.shift(1), crs=new_gdf.crs)
        ).fillna(0)).cumsum()
    
    combined_gdf = combined_gdf.append(new_gdf)

combined_gdf = combined_gdf.set_crs(crs=gdf.crs)

/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/dems/1970_09_29.tif
/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/dems/1974_08_10.tif
/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/dems/1977_09_27.tif
/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/dems/1991_09_09.tif
/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/dems/2013_09_13.tif
/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/dems/2015_09_01.tif
/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/rainbow/dems/2019_10_11.tif


In [8]:
combined_gdf.head()

,,id,n_from_glacial_max,geometry,centroid,coords,elevation,time,low_point_coords,path_distance
id,,,,,,,,,,
0,74,0,77,POINT (593136.896 5404570.235),POINT (593136.896 5404570.235),POINT (593136.9311215188 5404570.4327491),971.531006,1970-09-29,POINT (593136.9311215188 5404570.4327491),0.000000
1,261,1,76,POINT (593119.785 5404575.010),POINT (593119.785 5404575.010),POINT (593117.4454358442 5404563.117604766),971.505615,1970-09-29,POINT (593117.4454358442 5404563.117604766),17.765228
2,411,2,75,POINT (593105.169 5404592.810),POINT (593105.169 5404592.810),POINT (593097.0829522891 5404565.856589355),971.780029,1970-09-29,POINT (593097.0829522891 5404565.856589355),40.796812
3,506,3,74,POINT (593069.833 5404602.332),POINT (593069.833 5404602.332),POINT (593080.0787400091 5404615.520136361),972.104248,1970-09-29,POINT (593080.0787400091 5404615.520136361),77.393416
4,659,4,73,POINT (593050.578 5404624.338),POINT (593050.578 5404624.338),POINT (593062.2877953418 5404635.563351003),972.910889,1970-09-29,POINT (593062.2877953418 5404635.563351003),106.634004


In [9]:
def create_path_dist_from_glaciers(df):
    path_distance_at_glacier = df.loc[df['n_from_glacial_max']==0, 'path_distance'].iloc[0]
    df['path_distance_from_glacier'] = path_distance_at_glacier - df['path_distance']
    return df
combined_gdf = combined_gdf.groupby('time').apply(create_path_dist_from_glaciers)

In [10]:
combined_gdf.head()

,,id,n_from_glacial_max,geometry,centroid,coords,elevation,time,low_point_coords,path_distance,path_distance_from_glacier
id,,,,,,,,,,,
0,74,0,77,POINT (593136.896 5404570.235),POINT (593136.896 5404570.235),POINT (593136.9311215188 5404570.4327491),971.531006,1970-09-29,POINT (593136.9311215188 5404570.4327491),0.000000,1784.069905
1,261,1,76,POINT (593119.785 5404575.010),POINT (593119.785 5404575.010),POINT (593117.4454358442 5404563.117604766),971.505615,1970-09-29,POINT (593117.4454358442 5404563.117604766),17.765228,1766.304677
2,411,2,75,POINT (593105.169 5404592.810),POINT (593105.169 5404592.810),POINT (593097.0829522891 5404565.856589355),971.780029,1970-09-29,POINT (593097.0829522891 5404565.856589355),40.796812,1743.273093
3,506,3,74,POINT (593069.833 5404602.332),POINT (593069.833 5404602.332),POINT (593080.0787400091 5404615.520136361),972.104248,1970-09-29,POINT (593080.0787400091 5404615.520136361),77.393416,1706.676489
4,659,4,73,POINT (593050.578 5404624.338),POINT (593050.578 5404624.338),POINT (593062.2877953418 5404635.563351003),972.910889,1970-09-29,POINT (593062.2877953418 5404635.563351003),106.634004,1677.435901


# Mark points as (non)glacial

In [11]:
glaciers_gdf = gpd.read_file(glacier_polygons_file)
glaciers_gdf = glaciers_gdf.to_crs(combined_gdf.crs)
glaciers_gdf['time'] = glaciers_gdf['year'].apply(lambda d: datetime.datetime.strptime(d, strip_time_format))


In [12]:

combined_gdf['glacial'] = combined_gdf.apply(
    lambda row: any(glaciers_gdf.loc[glaciers_gdf['time'] == row["time"], 'geometry'].apply(lambda g: g.contains(row['coords']))),
    axis=1
)

Plot elevation profiles (large)

In [13]:

alt.Chart(
    combined_gdf[[ "time", "path_distance_from_glacier", "elevation", "glacial"]].reset_index()
).mark_line().encode(
    alt.X("path_distance_from_glacier:Q"),
    alt.Y("elevation:Q", scale=alt.Scale(zero=False), impute=alt.ImputeParams(value=None)),
    alt.Color("time:O"),
    alt.StrokeDash('glacial:N')
).properties(
    width = 1400,
    height = 600
)

alt.Chart(...)

Plot elevation profiles (small)

In [14]:

alt.Chart(
    combined_gdf[[ "time", "path_distance_from_glacier", "elevation", "glacial"]].reset_index()
).mark_line().encode(
    alt.X("path_distance_from_glacier:Q"),
    alt.Y("elevation:Q", scale=alt.Scale(zero=False), impute=alt.ImputeParams(value=None)),
    alt.Color("time:O"),
    alt.StrokeDash('glacial:N')
).properties(
    width = 600,
    # height = 600
)

alt.Chart(...)

# Calculate Residuals

Calculate so accumulation is always positive with time, erosion is negative with time.

In [15]:
diff_df = pd.DataFrame()

combined_gdf_grouped = combined_gdf.groupby("time")
reference_group = combined_gdf_grouped.get_group(reference_dem_date)

for timestamp, df in combined_gdf_grouped:    
    if timestamp != reference_dem_date:
        print(timestamp)
        this_diff_df = df.copy()
        merged = df.merge(reference_group, on='path_distance_from_glacier')
        if timestamp > reference_dem_date:
            residual_values = merged['elevation_x'] - merged['elevation_y']
        else:
            residual_values = merged['elevation_y'] - merged['elevation_x']
        assert len(this_diff_df) == len(residual_values)
        this_diff_df['elevation_residual'] = list(residual_values)
        diff_df = diff_df.append(this_diff_df)

1970-09-29 00:00:00
1974-08-10 00:00:00
1977-09-27 00:00:00
1991-09-09 00:00:00
2013-09-13 00:00:00
2019-10-11 00:00:00


Plot elevation residuals, exclude glacier signals (large)

In [16]:
src = diff_df[['elevation', 'time', 'path_distance_from_glacier', 'glacial', 'elevation_residual', 'n_from_glacial_max']].reset_index().dropna()
src['time'] = src['time'].apply(lambda x: x.strftime(strip_time_format))
alt.Chart(
    src
).transform_filter(
    (datum.glacial == False)
).mark_line().encode(
    alt.X("path_distance_from_glacier:Q"),
    alt.Y("elevation_residual:Q", scale=alt.Scale(zero=False), title='Elevation Residuals (rolling mean, 10 meter window', impute=alt.ImputeParams(value=None)),
    alt.Color("time:O", scale=alt.Scale(scheme='viridis')),
    alt.StrokeDash('glacial:N'),
).properties(
    width = 1400,
    height = 600,
    title="Elevation Residuals, relative to 2015 data."
).configure_legend(
    titleColor='black', 
    titleFontSize=12, 
    labelFontSize=16, 
    symbolStrokeWidth=4
)

alt.Chart(...)

In [17]:
src = diff_df[['elevation', 'time', 'path_distance_from_glacier', 'glacial', 'elevation_residual', 'n_from_glacial_max']].reset_index().dropna()
src['time'] = src['time'].apply(lambda x: x.strftime(strip_time_format))
alt.Chart(
    src
).transform_filter(
    (datum.glacial == False)
).mark_circle().encode(
    alt.X("path_distance_from_glacier:Q"),
    alt.Y("elevation_residual:Q", scale=alt.Scale(zero=False), title='Elevation Residuals (rolling mean, 10 meter window', impute=alt.ImputeParams(value=None)),
    alt.Color("time:O", scale=alt.Scale(scheme='viridis')),
    alt.StrokeDash('glacial:N'),
    tooltip=['n_from_glacial_max', 'time']

).properties(
    width = 1400,
    height = 600,
    title="Elevation Residuals, relative to 2015 data."
).configure_legend(
    titleColor='black', 
    titleFontSize=12, 
    labelFontSize=16, 
    symbolStrokeWidth=4
).interactive()

alt.Chart(...)

Plot elevation residuals, exclude glacier signals (small)

In [18]:
src = diff_df[['elevation', 'time', 'path_distance_from_glacier', 'glacial', 'elevation_residual']].reset_index().dropna()
src['time'] = src['time'].apply(lambda x: x.strftime(strip_time_format))
alt.Chart(
    src
).transform_filter(
    (datum.glacial == False)
).mark_line().encode(
    alt.X("path_distance_from_glacier:Q"),
    alt.Y("elevation_residual:Q", scale=alt.Scale(zero=False), title='Elevation Residuals (rolling mean, 10 meter window', impute=alt.ImputeParams(value=None)),
    alt.Color("time:O", scale=alt.Scale(scheme='viridis')),
    alt.StrokeDash('glacial:N')
).properties(
    width = 600,
    # height = 600,
    title="Elevation Residuals, relative to 2015 data."
).configure_legend(
    titleColor='black', 
    titleFontSize=12, 
    labelFontSize=16, 
    symbolStrokeWidth=4
)

alt.Chart(...)

Plot elevation residuals, exclude glacier signals, rolling mean (large)

In [19]:
src = diff_df[['elevation', 'time', 'path_distance_from_glacier', 'glacial', 'elevation_residual']].reset_index().dropna()
src['time'] = src['time'].apply(lambda x: x.strftime(strip_time_format))
alt.Chart(
    src
).transform_filter(
    (datum.glacial == False)
).transform_window(
    rolling_mean='mean(elevation_residual)',
    groupby=['time'],
    frame=[-5,5]
).mark_line().encode(
    alt.X("path_distance_from_glacier:Q"),
    alt.Y("rolling_mean:Q", scale=alt.Scale(zero=False), title='Elevation Residuals (rolling mean, 10 meter window', impute=alt.ImputeParams(value=None)),
    alt.Color("time:O", scale=alt.Scale(scheme='viridis')),
    alt.StrokeDash('glacial:N')
).properties(
    width = 1400,
    height = 600,
    title="Elevation Residuals, relative to 2015 data."
).configure_legend(
    titleColor='black', 
    titleFontSize=12, 
    labelFontSize=16, 
    symbolStrokeWidth=4
)

alt.Chart(...)

Plot elevation residuals, exclude glacier signals, rolling mean (small)

In [20]:
src = diff_df[['elevation', 'time', 'path_distance_from_glacier', 'glacial', 'elevation_residual']].reset_index().dropna()
src['time'] = src['time'].apply(lambda x: x.strftime(strip_time_format))
alt.Chart(
    src
).transform_filter(
    (datum.glacial == False)
).transform_window(
    rolling_mean='mean(elevation_residual)',
    groupby=['time'],
    frame=[-5,5]
).mark_line().encode(
    alt.X("path_distance_from_glacier:Q"),
    alt.Y("rolling_mean:Q", scale=alt.Scale(zero=False), title='Elevation Residuals (rolling mean, 10 meter window', impute=alt.ImputeParams(value=None)),
    alt.Color("time:O", scale=alt.Scale(scheme='viridis')),
    alt.StrokeDash('glacial:N')
).properties(
    width = 600,
    # height = 600,
    title="Elevation Residuals, relative to 2015 data."
).configure_legend(
    titleColor='black', 
    titleFontSize=12, 
    labelFontSize=16, 
    symbolStrokeWidth=4
)

alt.Chart(...)

Plot elevation residuals, include glacier signals (large)

In [21]:
src = diff_df[['elevation', 'time', 'path_distance_from_glacier', 'glacial', 'elevation_residual']].reset_index().dropna()
src['time'] = src['time'].apply(lambda x: x.strftime(strip_time_format))
alt.Chart(
    src
).mark_line().encode(
    alt.X("path_distance_from_glacier:Q"),
    alt.Y("elevation_residual:Q", scale=alt.Scale(zero=False), title='Elevation Residuals (rolling mean, 10 meter window', impute=alt.ImputeParams(value=None)),
    alt.Color("time:O", scale=alt.Scale(scheme='viridis')),
    alt.StrokeDash('glacial:N')
).properties(
    width = 1400,
    height = 600,
    title="Elevation Residuals, relative to 2015 data."
).configure_legend(
    titleColor='black', 
    titleFontSize=12, 
    labelFontSize=16, 
    symbolStrokeWidth=4
)

alt.Chart(...)

Plot elevation residuals, include glacier signals (small)

In [22]:
src = diff_df[['elevation', 'time', 'path_distance_from_glacier', 'glacial', 'elevation_residual']].reset_index().dropna()
src['time'] = src['time'].apply(lambda x: x.strftime(strip_time_format))
alt.Chart(
    src
).mark_line().encode(
    alt.X("path_distance_from_glacier:Q"),
    alt.Y("elevation_residual:Q", scale=alt.Scale(zero=False), title='Elevation Residuals (rolling mean, 10 meter window', impute=alt.ImputeParams(value=None)),
    alt.Color("time:O", scale=alt.Scale(scheme='viridis')),
    alt.StrokeDash('glacial:N')
).properties(
    width = 600,
    # height = 600,
    title="Elevation Residuals, relative to 2015 data."
).configure_legend(
    titleColor='black', 
    titleFontSize=12, 
    labelFontSize=16, 
    symbolStrokeWidth=4
)

alt.Chart(...)

Plot elevation residuals, include glacier signals, rolling mean (large)

In [23]:
src = diff_df[['elevation', 'time', 'path_distance_from_glacier', 'glacial', 'elevation_residual']].reset_index().dropna()
src['time'] = src['time'].apply(lambda x: x.strftime(strip_time_format))
alt.Chart(
    src
).transform_window(
    rolling_mean='mean(elevation_residual)',
    groupby=['time'],
    frame=[-5,5]
).mark_line().encode(
    alt.X("path_distance_from_glacier:Q"),
    alt.Y("rolling_mean:Q", scale=alt.Scale(zero=False), title='Elevation Residuals (rolling mean, 10 meter window', impute=alt.ImputeParams(value=None)),
    alt.Color("time:O", scale=alt.Scale(scheme='viridis')),
    alt.StrokeDash('glacial:N')
).properties(
    width = 1400,
    height = 600,
    title="Elevation Residuals, relative to 2015 data."
).configure_legend(
    titleColor='black', 
    titleFontSize=12, 
    labelFontSize=16, 
    symbolStrokeWidth=4
)

alt.Chart(...)

Plot elevation residuals, include glacier signals, rolling mean (small)

In [24]:
src = diff_df[['elevation', 'time', 'path_distance_from_glacier', 'glacial', 'elevation_residual']].reset_index().dropna()
src['time'] = src['time'].apply(lambda x: x.strftime(strip_time_format))
alt.Chart(
    src
).transform_window(
    rolling_mean='mean(elevation_residual)',
    groupby=['time'],
    frame=[-5,5]
).mark_line().encode(
    alt.X("path_distance_from_glacier:Q"),
    alt.Y("rolling_mean:Q", scale=alt.Scale(zero=False), title='Elevation Residuals (rolling mean, 10 meter window', impute=alt.ImputeParams(value=None)),
    alt.Color("time:O", scale=alt.Scale(scheme='viridis')),
    alt.StrokeDash('glacial:N')
).properties(
    width = 600,
    # height = 600,
    title="Elevation Residuals, relative to 2015 data."
).configure_legend(
    titleColor='black', 
    titleFontSize=12, 
    labelFontSize=16, 
    symbolStrokeWidth=4
)

alt.Chart(...)

# Calculate slope

In [25]:
combined_gdf.head()

,,id,n_from_glacial_max,geometry,centroid,coords,elevation,time,low_point_coords,path_distance,path_distance_from_glacier,glacial
id,,,,,,,,,,,,
0,74,0,77,POINT (593136.896 5404570.235),POINT (593136.896 5404570.235),POINT (593136.9311215188 5404570.4327491),971.531006,1970-09-29,POINT (593136.9311215188 5404570.4327491),0.000000,1784.069905,False
1,261,1,76,POINT (593119.785 5404575.010),POINT (593119.785 5404575.010),POINT (593117.4454358442 5404563.117604766),971.505615,1970-09-29,POINT (593117.4454358442 5404563.117604766),17.765228,1766.304677,False
2,411,2,75,POINT (593105.169 5404592.810),POINT (593105.169 5404592.810),POINT (593097.0829522891 5404565.856589355),971.780029,1970-09-29,POINT (593097.0829522891 5404565.856589355),40.796812,1743.273093,False
3,506,3,74,POINT (593069.833 5404602.332),POINT (593069.833 5404602.332),POINT (593080.0787400091 5404615.520136361),972.104248,1970-09-29,POINT (593080.0787400091 5404615.520136361),77.393416,1706.676489,False
4,659,4,73,POINT (593050.578 5404624.338),POINT (593050.578 5404624.338),POINT (593062.2877953418 5404635.563351003),972.910889,1970-09-29,POINT (593062.2877953418 5404635.563351003),106.634004,1677.435901,False


In [26]:
def calculate_gradient(df):
    df['slope'] = np.gradient(df['elevation'], df['path_distance_from_glacier'])
    return df

slope_df = combined_gdf.groupby('time').apply(lambda df: calculate_gradient(df))

Plot slope (large)

In [27]:
src = slope_df[['elevation', 'time', 'path_distance_from_glacier', 'glacial', 'slope']].reset_index().dropna()
src['time'] = src['time'].apply(lambda x: x.strftime(strip_time_format))
alt.Chart(
    src
).transform_filter(
    (datum.glacial == False)
).mark_line().encode(
    alt.X("path_distance_from_glacier:Q"),
    alt.Y("slope:Q", scale=alt.Scale(zero=False), title='Valley floor gradient (rolling mean, 30 meter window', impute=alt.ImputeParams(value=None)),
    alt.Color("time:O", scale=alt.Scale(scheme='viridis')),
    alt.StrokeDash('glacial:N')
).properties(
    width = 1400,
    height = 600,
    title="Valley floor gradient"
).configure_legend(
    titleColor='black', 
    titleFontSize=12, 
    labelFontSize=16, 
    symbolStrokeWidth=4
)

alt.Chart(...)

Plot slope, rolling mean (large)

In [28]:
src = slope_df[['elevation', 'time', 'path_distance_from_glacier', 'glacial', 'slope']].reset_index().dropna()
src['time'] = src['time'].apply(lambda x: x.strftime(strip_time_format))
alt.Chart(
    src
).transform_filter(
    (datum.glacial == False)
).transform_window(
    rolling_mean='mean(slope)',
    frame=[-5, 5],
    groupby=['time']
).mark_line().encode(
    alt.X("path_distance_from_glacier:Q"),
    alt.Y("rolling_mean:Q", scale=alt.Scale(zero=False), title='Valley floor gradient (rolling mean, 30 meter window', impute=alt.ImputeParams(value=None)),
    alt.Color("time:O", scale=alt.Scale(scheme='viridis')),
    alt.StrokeDash('glacial:N')
).properties(
    width = 1400,
    height = 600,
    title="Valley floor gradient"
).configure_legend(
    titleColor='black', 
    titleFontSize=12, 
    labelFontSize=16, 
    symbolStrokeWidth=4
)

alt.Chart(...)

Plot slope (small)

In [29]:
src = slope_df[['elevation', 'time', 'path_distance_from_glacier', 'glacial', 'slope']].reset_index().dropna()
src['time'] = src['time'].apply(lambda x: x.strftime(strip_time_format))
alt.Chart(
    src
).transform_filter(
    (datum.glacial == False)
).mark_line().encode(
    alt.X("path_distance_from_glacier:Q"),
    alt.Y("slope:Q", scale=alt.Scale(zero=False), title='Valley floor gradient (rolling mean, 30 meter window', impute=alt.ImputeParams(value=None)),
    alt.Color("time:O", scale=alt.Scale(scheme='viridis')),
    alt.StrokeDash('glacial:N')
).properties(
    width = 600,
    # height = 600,
    title="Valley floor gradient"
).configure_legend(
    titleColor='black', 
    titleFontSize=12, 
    labelFontSize=16, 
    symbolStrokeWidth=4
)

alt.Chart(...)

Plot slope, rolling mean (small)

In [30]:
src = slope_df[['elevation', 'time', 'path_distance_from_glacier', 'glacial', 'slope']].reset_index().dropna()
src['time'] = src['time'].apply(lambda x: x.strftime(strip_time_format))
alt.Chart(
    src
).transform_filter(
    (datum.glacial == False)
).transform_window(
    rolling_mean='mean(slope)',
    frame=[-5, 5],
    groupby=['time']
).mark_line().encode(
    alt.X("path_distance_from_glacier:Q"),
    alt.Y("rolling_mean:Q", scale=alt.Scale(zero=False), title='Valley floor gradient (rolling mean, 30 meter window', impute=alt.ImputeParams(value=None)),
    alt.Color("time:O", scale=alt.Scale(scheme='viridis')),
    alt.StrokeDash('glacial:N')
).properties(
    width = 600,
    # height = 600,
    title="Valley floor gradient"
).configure_legend(
    titleColor='black', 
    titleFontSize=12, 
    labelFontSize=16, 
    symbolStrokeWidth=4
)

alt.Chart(...)

## Group by kilometer upslope/downslope from glacier

In [31]:
compact = slope_df[['elevation', 'time', 'path_distance_from_glacier', 'glacial', 'slope']].reset_index().dropna()
compact = compact[~compact.glacial]
compact['kilometer downstream from glacier'] = compact['path_distance_from_glacier'].apply(lambda x: math.floor(x/group_slope_meters))
compact = compact.groupby(['time', 'kilometer downstream from glacier']).mean().reset_index()

In [32]:
alt.Chart(compact).mark_line(point=True).encode(
    alt.X('time:T'),
    alt.Y('slope:Q'),
    alt.Facet('kilometer downstream from glacier:O')
).properties(width=200)

alt.Chart(...)

# Export low points


In [33]:
combined_gdf.geometry = combined_gdf['low_point_coords']

combined_gdf_noglacial = combined_gdf.query("not glacial")

combined_gdf_noglacial[
    ['geometry', 'path_distance_from_glacier', 'elevation', 'id', 'time']
].reset_index(drop=True).to_file(
    output_lowpoints_file,
    driver="GeoJSON"
)

# Create streamlines from low points

In [34]:
from shapely.geometry import Point, LineString
streamlines = combined_gdf_noglacial.groupby("time").apply(lambda df: LineString([point for point in df.geometry.tolist() if point]))

In [35]:
streamlines_gdf = gpd.GeoDataFrame(geometry=streamlines, crs=combined_gdf.crs)

In [36]:
streamlines_gdf.to_file(output_streamlines_file)
